In [0]:
#!pip install findspark

You should consider upgrading via the '/databricks/python3/bin/python -m pip install --upgrade pip' command.


In [0]:
#!pip install pyspark

     |▍  

*** WARNING: max output size exceeded, skipping output. ***

     |████████████████████████████████| 281.3 MB 18 kB/s 
     |████████████████████████████████| 199 kB 47.9 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=a0ed7e4f1658dcd43334d57006439d86d65bc7f0a40239b364b9db70b408665d
  Stored in directory: /root/.cache/pip/wheels/1d/27/68/1382001655ef41217e1dd34d59aa777612135379bab64279e9
Successfully built pyspark
You should consider upgrading via the '/databricks/python3/bin/python -m pip install --upgrade pip' command.


In [0]:
!pip install ipython-sql

     |████████████████████████████████| 42 kB 1.8 MB/s 
     |████████████████████████████████| 1.6 MB 12.9 MB/s 
     |████████████████████████████████| 156 kB 57.1 MB/s 
  Created wheel for prettytable: filename=prettytable-0.7.2-py3-none-any.whl size=13699 sha256=9f2ede3e510d0c03e2b64c4bebc87e19305834c65080248c1a89c5653474a57e
  Stored in directory: /root/.cache/pip/wheels/48/6d/77/9517cb933af254f51a446f1a5ec9c2be3e45f17384940bce68
Successfully built prettytable
You should consider upgrading via the '/databricks/python3/bin/python -m pip install --upgrade pip' command.


In [0]:
import findspark
from pyspark.sql import SparkSession
from pyspark.context import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql import Row
import pyspark.sql.functions as F
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.streaming import StreamingContext
from pyspark.sql.functions import from_json
from pyspark.sql.functions import schema_of_json
import threading
from time import sleep
import time
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [0]:
#Initialisation de findspark
findspark.init()

In [0]:
spark = (SparkSession.builder.appName("Kafka Pyspark Streaming Learning")
        .master("local[*]")
        .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.0")
        .getOrCreate())
spark.sparkContext.setLogLevel("ERROR")

In [0]:
raw_json = spark.read.json("dbfs:/FileStore/tables/test.json", multiLine=True)

In [0]:
KAFKA_TOPIC_NAME = "vlib_status_ville"
KAFKA_BOOTSTRAP_SERVER = "51.38.185.58:9092"

df = (
spark.readStream.format("kafka")
.option("kafka.bootstrap.servers", KAFKA_BOOTSTRAP_SERVER)
.option("subscribe", KAFKA_TOPIC_NAME)
.option("startingOffsets", "latest")
.load()
.withColumn("value", from_json(col("value").cast("string"), raw_json.schema))\
.select(col('value.fields.name'), col('value.datasetid'), col('value.*'))\
)

print(df.schema)
print(df.isStreaming)

rep = df.select(col('name').alias('name'),
                col('fields.mechanical').alias('mech'),
                col('fields.is_renting').alias('isrent'))
    

query = rep.filter(rep.isrent == 'OUI').groupBy('name').avg('mech') \
    .writeStream \
    .outputMode("complete") \
    .format("memory") \
    .queryName("memory_spark")\
    .start()

while(query.isActive):
    time.sleep(60)
    spark.sql("select * from memory_spark").show()


query.awaitTermination()

StructType(List(StructField(name,StringType,true),StructField(datasetid,StringType,true),StructField(datasetid,StringType,true),StructField(fields,StructType(List(StructField(capacity,LongType,true),StructField(coordonnees_geo,ArrayType(DoubleType,true),true),StructField(duedate,StringType,true),StructField(ebike,LongType,true),StructField(is_installed,StringType,true),StructField(is_renting,StringType,true),StructField(is_returning,StringType,true),StructField(mechanical,LongType,true),StructField(name,StringType,true),StructField(nom_arrondissement_communes,StringType,true),StructField(numbikesavailable,LongType,true),StructField(numdocksavailable,LongType,true),StructField(stationcode,StringType,true))),true),StructField(geometry,StructType(List(StructField(coordinates,ArrayType(DoubleType,true),true),StructField(type,StringType,true))),true),StructField(record_timestamp,StringType,true),StructField(recordid,StringType,true)))
True
+--------------------+---------+
|                n

In [0]:
%sql select * from memory_spark